In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

#### Read the file

In [3]:
books_engineered = pd.read_csv('data_algo/books_engineered.csv', index_col=0)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
reader_clusters = books_engineered.copy()

In [5]:
reader_clusters = reader_clusters[reader_clusters['id_book'].notna() == True]

In [6]:
reader_clusters['reader_book_id'] = reader_clusters['id_reader'].astype('str') + reader_clusters['id_book'].astype('str')

In [7]:
reader_clusters.drop_duplicates(['reader_book_id'], inplace=True)

In [8]:
reader_clusters.head()

,id_issue,id_reader,id_book,author,name,reader_age,genre,is_novice,is_old,is_series,is_age_cencored,is_18_cencored,reader_book_id
0,1,179,5190.0,Семенова Мария Васильевна,Год Людоеда,30.0,russian_fiction,N,N,N,N,N,1795190.0
1,2,179,5185.0,NaN,Сказки о солдате,30.0,poetry_folklor,N,N,Y,N,N,1795185.0
7,8,163,5190.0,Семенова Мария Васильевна,Год Людоеда,38.0,russian_fiction,N,N,N,N,N,1635190.0
9,10,232,5456.0,Андреева Наталья Вячеславовна,Москва не принимает,80.0,russian_fiction,N,N,Y,N,N,2325456.0
10,11,232,5324.0,Акунин Борис,Пелагия и белый бульдог,80.0,russian_fiction,N,N,Y,N,N,2325324.0


In [9]:
reader_book = reader_clusters[['id_reader', 'id_book']]

#### Adjust if necessary

In [10]:
reader_book['id_reader'] = reader_book['id_reader'].astype(int)
reader_book['id_book'] = reader_book['id_book'].astype(int)

c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
c:\users\vpoletae\appdata\local\continuum\anaconda3\envs\mytfenv\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
user_map = dict()
back_user_map = dict()
counter = 0
for user in reader_book['id_reader'].unique().tolist():
    user_map[user] = counter
    back_user_map[counter] = user
    counter += 1

In [12]:
items_map = dict()
back_items_map = dict()
counter = 0
for item in reader_book['id_book'].unique().tolist():
    items_map[item] = counter
    back_items_map[counter] = item
    counter += 1

In [13]:
%%time
rows = []
cols = []
data = []
for row in reader_book.itertuples():
    rows.append(user_map[row[1]]) #! check
    cols.append(items_map[row[2]])
    data.append(1)

Wall time: 9.65 s


In [14]:
row = np.array(rows)
col = np.array(cols)
data = np.array(data)
user_item_matrix = csr_matrix((data, (row, col)), shape=(len(user_map.keys()), len(items_map.keys())))

#### Apply the model

In [ ]:
%%time
model = TruncatedSVD(n_components = 2000, algorithm='randomized', n_iter=5)
model.fit(user_item_matrix)

In [ ]:
model.explained_variance_ratio_.sum()

In [ ]:
def predict_services(user_id, top_n): 
    user_interactions = user_item_matrix.getrow(user_map[user_id])
    user_low_dimensions = model.transform(user_interactions)
    return get_recommendations(user_low_dimensions, user_interactions, user_id, top_n)

In [ ]:
def get_recommendations(dimensions, interactions, id_, top_n=1):
    recommendations = []
    predictions = model.inverse_transform(dimensions)[0]
    for index in reversed(np.argsort(predictions)):
        try:
            if interactions[0, index] == 0:
                recommendations.append((id_, back_items_map[index]))
        except IndexError:
            pass

    return recommendations[:top_n]